In [1]:
!pip install deep-translator

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#импортирую модели переводчиков
from deep_translator import GoogleTranslator, YandexTranslator, DeeplTranslator

In [3]:
import pandas as pd
import requests
import json
import numpy as np

In [5]:
#импортирую токенизатор
import nltk
from nltk.tokenize import word_tokenize

### Обработка исходного датасета

In [37]:
#создаю список для токенизированных предложений из датасета
sents = []

In [ ]:
#записываю в sents обработанные предложения
file = open("RULEC-GEC.test.sents", "r")
while True:    
    line = file.readline()
    if len(line.split('[')) > 1:
        line = line.split('[')[1]
        sents.append(line.strip())
    if not line:
        break 
    print(line.strip())
file.close()

### Функция двойного перевода для GoogleTranslator

In [53]:
def double_tr_google(lang, sents):
    fixed = []
    for s in sents:
        try:
            sht = GoogleTranslator(source='auto', target='ru').translate(GoogleTranslator(source='auto', target=lang)
                                                                         .translate(s))
            fixed.append(sht)
        #в случае возникновения ограничений или сбоя в работе переводчика код выдаст 'Translation Error'
        except Exception:
            fixed.append('Translation Error')
    return fixed

### Функция двойного перевода для Яндекс.Переводчика

Для бесплатного использования Яндекс.Переводчика нужно получить IAM_TOKEN и folder_id, сделать это можно при наличии аккаунта на Yandex Cloud - функция "double_tr_yandex_1".

Иной вариант: получить API-key (выдается платно) и использовать deep-translator - функция "double_tr_yandex_2".

In [54]:
#Время жизни IAM-токена — не больше 12 часов, но рекомендуется запрашивать его чаще, например каждый час.
IAM_TOKEN = 'your_IAM_TOKEN'
folder_id = 'your_folder_id'

In [55]:
def double_tr_yandex_1(lang, sents):
    fixed = []
    headers = {
            "Content-Type": "application/json",
            "Authorization": "Bearer {0}".format(IAM_TOKEN)
        }
    for s in sents:
        try:
            response_1 = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate',
                json = {
                    "sourceLanguageCode": 'ru',
                    "targetLanguageCode": lang,
                    "texts": s,
                    "folderId": folder_id,
                },
                headers = headers
            )
            response_2 = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate',
                json = {
                    "sourceLanguageCode": lang,
                    "targetLanguageCode": 'ru',
                    "texts": json.loads(response_1.text)['translations'][0]['text'],
                    "folderId": folder_id,
                },
                headers = headers
            )
            fixed.append(json.loads(response_2.text)['translations'][0]['text'])
        #в случае возникновения ограничений или сбоя в работе переводчика код выдаст 'Translation Error'    
        except Exception:
            fixed.append('Translation Error')
    return fixed

In [51]:
def double_tr_yandex_2(lang, sents):
    fixed = []
    for s in sents:
        try:
            to_lang = YandexTranslator('your_api_key').translate(source='auto', target=lang, text=s)
            from_lang = YandexTranslator('your_api_key').translate(source=lang, target='ru', text=to_lang)
            fixed.append(from_lang)
        #в случае возникновения ограничений или сбоя в работе переводчика код выдаст 'Translation Error'
        except Exception:
            fixed.append('Translation Error')
    return fixed

### Функция двойного перевода для DeepL

In [52]:
def double_tr_DeepL(lang, sents):
    fixed = []
    for s in sents:
        try:
            to_lang = DeeplTranslator(api_key='your_api_key', source='ru', target=lang, use_free_api=True).translate(s)
            from_lang = DeeplTranslator(api_key='your_api_key', source=lang, target='ru', use_free_api=True).translate(to_lang)
            fixed.append(from_lang)
        #в случае возникновения ограничений или сбоя в работе переводчика код выдаст 'Translation Error'
        except Exception:
            fixed.append('Translation Error')
    return fixed

#### Пример работы функций

In [56]:
print(double_tr_google('en', sents[:2]))
print(double_tr_yandex_1('en', sents[2:4]))
print(double_tr_yandex_2('en', sents[4:6]))
print(double_tr_DeepL('en', sents[6:8]))

['Экологическая составляющая ситуации Аральского моря проявляется в нескольких формах.', 'Смысл всегда выражен ясно, но термины и предложения очень сложны.']
['Я продолжаю бродить по музею и в какой-то момент понимаю, что я больше не один.', 'Мы видели, насколько хорошо все обернулось с ипотекой в условиях нашего нынешнего кризиса.']
['Со стороны этой прекрасной вещи я понял, что как только она вышла замуж и стала женой фараона, царицей, появилась Соня Пароходова.', 'План выпуска новостей :']
['Однако здесь есть проблема - возможность пользоваться компьютерными программами, а в некоторых селах, похоже, нет даже доступа к Интернету.', 'Поскольку Толстой работал в университете, она больше всего общалась с интеллигентными людьми, которые очень приятны и не любят обижать других.']


#### Пример применения метода двойного перевода (язык-посредник - белорусский)

In [57]:
result_bel = []
#применение функции двойного перевода
result_bel = double_tr_google('belarusian', sents)
bel_tokenized = []
#токенизация предложений
for sht in result_bel:
    text = word_tokenize(sht)
    text = ' '.join(text)
    bel_tokenized.append(text)
#запись токенизированных предложений в файл
with open('bel_tokenized_google', 'w') as file:
    for item in bel_tokenized:
        file.write("%s\n" % item)

### Подготовка файлов для анализа по типам ошибок

In [61]:
#получение тэгов ошибок
tags = []
with open('RULEC-GEC.test.M2') as file:
    for line in file:
        if line.startswith('A '):
            tags.append(line.split('|||')[1])
tags = sorted(list(set(tags)))
tags[6] = 'Глагол:Число_Лицо'


{'Прил.:Др.', 'Заменить', 'Параллель', 'Сущ.:Др.', 'Предлог', 'Обр.параллель', 'Сущ.:Род', 'Сущ.:Падеж', 'Сущ.:Число', 'Лексика:замена', 'Орфография', 'Глагол:Вид', 'Прил.:Число', 'Глагол:Залог', 'Местоимение', 'Убрать', 'Глагол:Число/Лицо', 'Глагол:Др', 'Прил.:Род', 'Вставить', 'Лексика:морф.', 'Глагол:Время', 'Сущ:число', 'Прил.:Падеж', 'Союз', 'Глагол:Др.'}


In [65]:
#деление 'RULEC-GEC.test.M2' на файлы с отдельными типами ошибки
with open('RULEC-GEC.test.M2') as file:
    text = file.readlines()
    for i in tags:
        sp = []
        for line in text:
            if line.startswith('A '):
                if i in line:
                    sp.append(line)
            else:
                sp.append(line)
        name_f = 'RULEC-GEC_' + i + '.test.M2'
        with open(name_f, 'w') as f:
            for j in sp:
                f.write(j)                        

### Подготовка сводных датасетов по ошибкам группы С для анализа

#### Пример кода для типа ошибки "Вставить"

In [66]:
#выбор предложений с нужным типом ошибки и разметки к ним
with open('RULEC-GEC_Вставить.test.M2') as file:
    i = -1
    corr = {}
    sents_corr = []
    for line in file:
        if line.startswith('S '):
            i += 1
        elif line.startswith('A '):
            sents_corr.append(i)
            if i not in corr.keys():
                corr[i] = [line.strip()]
            else:
                corr[i].append(line.strip())
chosen = sorted(set(sents_corr))

#список с названиями файлов по языкам-посредникам
transl = ['fixed_eng.txt', 'fixed_fr.txt', 'fixed_cz.txt', 'bel_tokenized.txt', 'fin_tokenized.txt', 'hin_tokenized.txt', 'turk_tokenized.txt']
slov = {}
#создание выборки исправленных предложений по нужному типу ошибки
for sht in transl:
    name = sht.split('.')[0]
    slov[name] = []
    with open(sht) as file:
        for line in file:
            slov[name].append(line.strip())
    slov[name] = np.array(slov[name])[chosen]

slov['sents'] = np.array(sents)[chosen]
#добавление разметки к выборке
corrections = []
for value in corr.values():
    corrections.append('\n'.join(value))
slov['corrections'] = corrections            

In [67]:
#преобразование словаря в таблицу
data = pd.DataFrame(slov)

In [68]:
data

,fixed_eng,fixed_fr,fixed_cz,bel_tokenized,fin_tokenized,hin_tokenized,turk_tokenized,sents,corrections
0,"Когда я впервые прочитал их разговор , мне при...","Когда я впервые прочитал их разговор , мне при...","В первый раз , когда я прочитал их разговор , ...","Когда я впервые прочитал их разговор , мне при...","Когда я впервые прочитал их разговор , мне при...","Когда я впервые прочитал их разговор , мне при...","Когда я впервые прочитал их разговор , мне при...","Когда я первый раз читала их разговор , мне бы...","A 21 21|||Вставить|||,|||REQUIRED|||-NONE-|||0"
1,Но самая известная и крупная волна – это « вто...,Но самая известная и важная волна — это « втор...,Но самая известная и крупная волна – это « вто...,Но самая известная и крупная волна – это « вто...,Но самая известная и крупная волна – это « вто...,Но самая известная и крупная волна – это « вто...,Но самая известная и крупная волна — это « вто...,"Но самой известной и большой волной является ""...",A 18 18|||Вставить|||-|||REQUIRED|||-NONE-|||0...
2,"Вера сказала ему , что обдумала его предложени...","Вера сказала ему , что обдумала его предложени...","Вера сказала ему , что обдумала его предложени...","Вера сказала ему , что думала над его предложе...","Вера сказала ему , что обдумала его предложени...","Вера сказала ему , что обдумала его предложени...","Вера сказала ему , что рассматривает его предл...","Вера сказала ему , что думала о его предложени...",A 11 11|||Вставить|||в|||REQUIRED|||-NONE-|||0
3,Римляне прорезали прямые дороги от А до Б и см...,Римляне проложили прямые дороги от А до Б и пр...,"Римляне прорезали прямые дороги от А до Б , см...",Римляне прокладывали прямые дороги из пункта А...,Римляне прорезали прямые дороги от А до Б и см...,Римляне прорезали прямые дороги от А до Б и см...,Римляне прорезали прямые дороги от А до Б и по...,"Римляне вырубали прямые дороги , из пункта А в...","A 16 16|||Вставить|||,|||REQUIRED|||-NONE-|||0"
4,Автор особенно подробно рассматривает предложе...,"В частности , автор рассматривает предложение ...",Автор подробно рассматривает предложение о соз...,Особенно подробно автор рассматривает предложе...,Автор особенно подробно рассматривает предложе...,Автор рассматривает предложение о создании общ...,Автор особенно подробно рассматривает предложе...,Автор особенно детально рассматривает предложе...,A 5 5|||Вставить|||о|||REQUIRED|||-NONE-|||0
...,...,...,...,...,...,...,...,...,...
750,"В прошлом , когда я ел в таком ресторане , мне...","Раньше , когда я ел в таком ресторане , еда мн...","Раньше , когда я ел в таком ресторане , мне не...","Раньше , когда я ел в этом ресторане , мне не ...","В прошлом , когда я ел в таком ресторане , мне...","В прошлом , когда я ел в таких ресторанах , мн...","Раньше , когда я ел в таком ресторане , мне не...","В прошлом когда я ела в таком ресторане , я не...","A 2 2|||Вставить|||,|||REQUIRED|||-NONE-|||0"
751,"Я , наверное , мог бы , на мой взгляд , уехать...","Я , наверное , мог бы , на мой взгляд , навсег...","Я , наверное , мог бы , на мой взгляд , навсег...","Я , наверное , мог бы , на мой взгляд , навсег...","Я , наверное , думаю , что мог бы уехать в дру...","Я , наверное , мог бы , по-моему , навсегда уе...","Думаю , я мог бы уехать в другую страну навсег...","Я наверное смогла бы , по-моему , уехать в дру...","A 1 1|||Вставить|||,|||REQUIRED|||-NONE-|||0\n..."
752,"У всех примеров есть что-то общее , они начина...","У всех примеров есть что-то общее , они начина...","У всех примеров есть что-то общее , начинали о...",У всех примеров есть одна общая черта : они на...,"У всех примеров есть что-то общее , они были м...","У всех примеров есть что-то общее , они начина...","У всех примеров есть одна общая черта , они на...","У всех примеров есть общие черты , они начали ...",A 23 23|||Вставить|||товаров или услуг|||REQUI...
753,"Большинство мексиканцев , пересекших границу ,...","Большинство мексиканцев , пересекших границу ,...","Большинство мексиканцев , пересекших границу ,...

In [69]:
#запись таблицы в файл
data.to_csv('Add.csv')